In [1]:
import os
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession
import warnings

warnings.filterwarnings("ignore")

# Encontrar o caminho absoluto do diretório atual
notebook_cwd = os.getcwd()

# Definir o caminho correto para a raiz do projeto Kedro
project_path = r"c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project"

# Verificar o diretório atual e o caminho do projeto
print(f"Notebook current working directory: {notebook_cwd}")
print(f"Project path: {project_path}")

# Alterar para o diretório raiz do projeto Kedro
os.chdir(project_path)

# Bootstrap o projeto Kedro
bootstrap_project(project_path)

# Inicialize o contexto do Kedro
with KedroSession.create() as session:
    context = session.load_context()

# Recarregar o catálogo
catalog = context.catalog

# Acessar os parâmetros
params = context.params

# Listar o catálogo
catalog.list()

[09/15/24 01:08:29] INFO     Using                                                                  ]8;id=736777;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=409111;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framework\project\__init__.py#249\249]8;;\
                             'c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\framew                
                             ork\project\rich_logging.yml' as logging configuration.                               

Notebook current working directory: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\notebooks\gmferratti\pipeline
Project path: c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project


[09/15/24 01:08:31] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=235503;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py\plugin.py]8;;\:]8;id=48279;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro_telemetry\plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         


[
    'decks_json_partitioned',
    'decks_txt_partitioned',
    'sampled_decks',
    'players',
    'players_with_decks',
    'parameters',
    'params:global',
    'params:global.run_date',
    'params:global.run_date.day',
    'params:global.run_date.month',
    'params:global.run_date.year',
    'params:global.user',
    'params:global.user.project_path',
    'params:preprocessing',
    'params:preprocessing.webscraper',
    'params:preprocessing.webscraper.zip_url',
    'params:preprocessing.webscraper.zip_folder',
    'params:preprocessing.webscraper.log_folder',
    'params:preprocessing.webscraper.deck_cards',
    'params:preprocessing.webscraper.sample_size_ratio',
    'params:simulation',
    'params:simulation.n_players',
    'params:simulation.matches_per_player',
    'params:simulation.hand_size_stop',
    'params:simulation.max_turns',
    'params:simulation.max_mulligans',
    'params:simulation.mulligan_prob',
    'params:simulation.extra_land_prob',
    'params:simula

In [2]:
from typing import List, Dict
from classes.deck import Deck
from classes.player import Player
from classes.player_tracker import PlayerTracker
from src.mtg_project.pipelines.utils import setup_logger
import pandas as pd
from faker import Faker
import random

In [3]:
def create_players(n_players: int):
    """
    Cria uma lista de objetos Player com nomes aleatórios.

    Args:
        n_players (int): Número de jogadores a serem criados.

    Returns:
        List[Player]: Lista de objetos Player com nomes gerados aleatoriamente.
    """
    # Inicializando o gerador de dados falsos Faker
    fake = Faker()
    
    # Gerando uma lista de nomes aleatórios usando o Faker
    player_names = [fake.first_name() + " " + fake.last_name() for _ in range(n_players)]
    
    # Criando uma lista de objetos Player a partir dos nomes gerados
    players = [Player(name) for name in player_names]

    # Retornando a lista de objetos Player
    return players

n_players = catalog.load("params:simulation.n_players")
players = create_players(n_players)
catalog.save("players", players)

[09/15/24 01:08:32] INFO     Loading data from params:simulation.n_players (MemoryDataset)...   ]8;id=981396;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=450714;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

[09/15/24 01:08:33] INFO     Saving data to players (PickleDataset)...                          ]8;id=352374;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=774061;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#581\581]8;;\

In [4]:
import random
import os
from typing import List, Dict

def assign_decks_to_players(
        players: List[Player], 
        sampled_decks: Dict[str, str],
        log_folder: str) -> List[Player]:
    """
    Função para atribuir decks aleatórios a cada player na lista de players.

    A função tentará atribuir um deck a cada player chamando o método assign_deck().
    Caso ocorra algum erro na atribuição, tentará com outro deck disponível.

    Args:
        players (list): Lista de objetos Player.
        sampled_decks (dict): Dicionário com os nomes e caminhos dos decks.
        log_folder (str): Caminho da pasta para salvar o log.

    Returns:
        List[Player]: Lista de objetos Player com decks atribuídos.
    """
    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'decks_assignment.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("validate_decks", log_filepath)
    
    # Log de início da validação
    logger.info("Validating decks...")

    # Convertemos as chaves do dicionário para uma lista de nomes de decks disponíveis
    available_decks = list(sampled_decks.keys())
    
    for player in players:
        assigned = False
        while not assigned and available_decks:
            try:
                # Seleciona um deck aleatório da lista de decks disponíveis
                deck_name = random.choice(available_decks)

                # Obter o caminho completo do deck a partir do dicionário sampled_decks
                deck_path = sampled_decks[deck_name]

                # Cria um novo objeto Deck
                deck = Deck()

                # Carrega o deck a partir do arquivo .txt no caminho obtido
                deck.load_deck_from_txt(deck_path)

                # Atribui o deck ao player
                player.assign_deck(deck)
                logger.info(f"Deck '{deck_name}' assigned to player '{player.name}'")
                
                # Remove o deck da lista de decks disponíveis para evitar reutilização
                available_decks.remove(deck_name)

                assigned = True  # Deck atribuído com sucesso
            except Exception as e:
                # Em caso de erro, tenta outro deck
                logger.error(f"Failed to assign deck '{deck_name}' to player '{player.name}': {e}")
                continue
        
        
        # Se não houver mais decks disponíveis e não conseguir atribuir, lança um erro
        if not assigned:
            raise ValueError(f"No available decks left to assign to player '{player.name}'.")

    logger.info("Deck assignment process completed.")

    return players

players = catalog.load("players")
sampled_decks = catalog.load("sampled_decks")
log_folder = catalog.load("params:simulation.log_folder")
# players_with_decks = assign_decks_to_players(players, sampled_decks, log_folder)
# catalog.save("players_with_decks", players_with_decks)

                    INFO     Loading data from players (PickleDataset)...                       ]8;id=736018;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=145141;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from sampled_decks (PickleDataset)...                 ]8;id=170401;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=608295;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from params:simulation.log_folder (MemoryDataset)...  ]8;id=566451;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=498044;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

In [5]:
def simulate_player_matches(params: dict, players_with_decks: list) -> pd.DataFrame:
    """
    Simulates Magic: The Gathering matches for a list of players based on the provided simulation parameters.

    Parameters:
    -----------
    params : dict
        A dictionary containing the simulation parameters, including:
        - 'max_mulligans': Maximum number of mulligans allowed per player.
        - 'mulligan_prob': Probability of a player choosing to mulligan.
        - 'hand_size_stop': Minimum hand size at which the simulation will stop.
        - 'max_turns': Maximum number of turns per match.
        - 'extra_land_prob': Probability of playing an extra land during a turn.
        - 'matches_per_player': Number of matches to simulate per player.
        - 'log_folder': Folder path for logging the simulation process.
    
    players_with_decks : list
        A list of Player objects, each with an assigned deck to be used in the simulation.

    Returns:
    --------
    pd.DataFrame
        A DataFrame containing the match data for all players across all matches and turns, including:
        - Player attributes at each turn.
        - Match number for each simulation.
    """
    
    # Atribuir os parâmetros
    max_mulligans = params["max_mulligans"]
    mulligan_prob = params["mulligan_prob"]
    hand_size_stop = params["hand_size_stop"]
    max_turns = params["max_turns"]
    extra_land_prob = params["extra_land_prob"]
    matches_per_player = params["matches_per_player"]
    log_folder = params["log_folder"]

    # Caminho do arquivo de log
    log_filepath = os.path.join(log_folder, 'player_matches.txt')

    # Cria a pasta de log se ela não existir
    os.makedirs(log_folder, exist_ok=True)

    # Configura o logger geral
    logger = setup_logger("player_matches", log_filepath)

    # Log de início da validação
    logger.info("Initiating simulations...")

    # Inicializa o tracker para armazenar os dados
    tracker = PlayerTracker()

    # Loop através dos jogadores e realizar as simulações de partidas
    for player in players_with_decks:
        for match in range(matches_per_player):
            # Simular várias partidas para o jogador
            player.play_a_match(tracker, 
                                max_mulligans, 
                                mulligan_prob, 
                                max_turns, 
                                hand_size_stop, 
                                extra_land_prob)

    # Obter os dados de todas as partidas e turnos
    matches_dataframe = tracker.get_data()
    
    return matches_dataframe

# Chamada da função
params = catalog.load("params:simulation")
players_with_decks = catalog.load("players_with_decks")
matches_dataframe = simulate_player_matches(params, players_with_decks)
matches_dataframe

                    INFO     Loading data from params:simulation (MemoryDataset)...             ]8;id=980156;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=694011;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    INFO     Loading data from players_with_decks (PickleDataset)...            ]8;id=917692;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=929100;file://c:\Users\gufer\miniconda3\envs\mtg_env\Lib\site-packages\kedro\io\data_catalog.py#539\539]8;;\

                    WARNING  Peter Perry couldn't play any spells.                                    ]8;id=741060;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=234929;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

                    WARNING  Peter Perry couldn't play any spells.                                    ]8;id=155434;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=575472;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

                    WARNING  Peter Perry couldn't play any spells.                                    ]8;id=620661;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=34665;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

...

                    WARNING  Philip Phillips couldn't play any spells.                                ]8;id=238903;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=366693;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

                    WARNING  Philip Phillips couldn't play any spells.                                ]8;id=302477;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=646191;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

                    WARNING  Philip Phillips couldn't play any spells.                                ]8;id=831813;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=905881;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

                    WARNING  Philip Phillips couldn't play any spells.                                ]8;id=453375;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py\player.py]8;;\:]8;id=210090;file://c:\Users\gufer\OneDrive\Documentos\FIAP\Fase_03\mtg-project\classes\player.py#366\366]8;;\

,name,deck_name,deck_colors,turn,mulligan_count,lands_played,spells_played,extra_lands,mana_pool,hand_size,library_size,graveyard_size,full_hand,full_graveyard,match
0,Peter Perry,inspirationstruck,"{U, B}",0,0,0,0,0,0,7,53,0,"Hand(7 cards: Griptide, Thassa's Bounty, Forlo...",Graveyard(0 cards: ),1
1,Peter Perry,inspirationstruck,"{U, B}",1,0,0,0,0,0,7,52,1,"Hand(7 cards: Griptide, Forlorn Pseudamma, Aer...",Graveyard(1 cards: Thassa's Bounty),1
2,Peter Perry,inspirationstruck,"{U, B}",2,0,0,0,0,0,7,51,2,"Hand(7 cards: Griptide, Forlorn Pseudamma, Aer...","Graveyard(2 cards: Thassa's Bounty, Warchanter...",1
3,Peter Perry,inspirationstruck,"{U, B}",3,0,0,0,0,0,7,50,3,"Hand(7 cards: Griptide, Forlorn Pseudamma, Aer...","Graveyard(3 cards: Thassa's Bounty, Warchanter...",1
4,Peter Perry,inspirationstruck,"{U, B}",4,0,1,0,0,1,7,49,3,"Hand(7 cards: Griptide, Forlorn Pseudamma, Aer...","Graveyard(3 cards: Thassa's Bounty, Warchanter...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333,Philip Phillips,forests might 2,{G},1,0,1,1,0,1,6,52,1,"Hand(6 cards: Karplusan Forest, Karplusan Fore...",Graveyard(1 cards: Giant Growth),10
334,Philip Phillips,forests might 2,{G},2,0,1,2,0,2,5,51,2,"Hand(5 cards: Karplusan Forest, Karplusan Fore...","Graveyard(2 cards: Giant Growth, Growth Cycle)",10
335,Philip Phillips,forests might 2,{G},3,0,1,3,0,3,4,50,3,"Hand(4 cards: Karplusan Forest, Karplusan Fore...","Graveyard(3 cards: Giant Growth, Growth Cycle,...",10
336,Philip Phillips,forests might 2,{G},4,0,1,5,0,4,2,49,5,"Hand(2 cards: Karplusan Forest, Aggressive Mam...","Graveyard(5 cards: Giant Growth, Growth Cycle,...",10


In [7]:
# extra lands seems not to be working properly
matches_dataframe[matches_dataframe["lands_played"] > 1]

,name,deck_name,deck_colors,turn,mulligan_count,lands_played,spells_played,extra_lands,mana_pool,hand_size,library_size,graveyard_size,full_hand,full_graveyard,match
25,Peter Perry,inspirationstruck,"{U, B}",5,0,2,5,0,6,1,48,5,Hand(1 cards: Horizon Scholar),"Graveyard(5 cards: Retraction Helix, Shipwreck...",3
30,Peter Perry,inspirationstruck,"{U, B}",4,0,2,2,0,5,4,49,2,"Hand(4 cards: Warchanter of Mogis, Griptide, T...","Graveyard(2 cards: Felhide Minotaur, Aerie Wor...",4
87,Peter Perry,inspirationstruck,"{U, B}",4,0,2,4,0,5,2,49,4,"Hand(2 cards: Island, Deepwater Hypnotist)","Graveyard(4 cards: Springleaf Drum, Retraction...",10
135,Kimberly Coleman,war of attrition,{W},8,0,2,6,0,9,0,45,6,Hand(0 cards: ),"Graveyard(6 cards: Flayer Husk, Journey to Now...",7
137,Kimberly Coleman,war of attrition,{W},1,0,2,1,0,2,5,52,1,"Hand(5 cards: Plains, Plains, Leonin Skyhunter...",Graveyard(1 cards: Elite Vanguard),8
212,Brooke Jackson,ajani valiant protector,"{W, G}",4,0,2,2,0,5,4,49,2,"Hand(4 cards: Karplusan Forest, Plains, Lifecr...","Graveyard(2 cards: Verdant Automaton, Inspirin...",6
213,Brooke Jackson,ajani valiant protector,"{W, G}",5,0,2,3,0,7,2,48,3,"Hand(2 cards: Plains, Engineered Might)","Graveyard(3 cards: Verdant Automaton, Inspirin...",6
221,Brooke Jackson,ajani valiant protector,"{W, G}",6,0,2,6,0,7,0,47,6,Hand(0 cards: ),"Graveyard(6 cards: Narnam Renegade, Ajani's Co...",7
295,Philip Phillips,forests might 2,{G},3,0,2,2,0,4,4,50,2,"Hand(4 cards: Wakeroot Elemental, Season of Gr...","Graveyard(2 cards: Barkhide Troll, Oakenform)",5
302,Philip Phillips,forests might 2,{G},2,0,2,2,0,3,4,51,2,"Hand(4 cards: Growth Cycle, Giant Growth, Vora...","Graveyard(2 cards: Giant Growth, Growth Cycle)",6
